# 명탐정 코난 매니아 판별기 확장

## 애플리케이션 기본 정보 설정

### `config.py`: 구성정보

#### `Path`

- `Path(__file__)`: 현재 실행 중인 소스 파일의 경로(스크립트 파일의 물리적 위치)를 기준 - Jupyter Notebook에서는 `__file__` 이 정의되지 않음.
- `Path.cwd()`: 프로그램 실행 시점의 현재 작업 디렉토리 (터미널 폴더 위치)

#### `BaseSettings`

- 환경 변수(`.env` 포함) 또는 인자값을 바탕으로 설정 클래스를 정의할 때 사용하는 기본 클래스
- `BaseSettings` 를 상속받아 만든 클래스는 자동으로 환경 변수/인자에서 값을 읽어와 필드를 채워줌.

In [3]:
from pathlib import Path
from dotenv import load_dotenv
from pydantic import Field, computed_field
from pydantic_settings import BaseSettings

# .env 로드
load_dotenv()

# __file__ 이 없는 경우 (예: Jupyter) 대비 fallback
try:
    DEFAULT_BASE_DIR = Path(__file__).resolve().parent
except NameError:
    DEFAULT_BASE_DIR = Path.cwd()


class Settings(BaseSettings):
    # 기본 경로
    BASE_DIR: Path = Field(default_factory=lambda: DEFAULT_BASE_DIR)

    # 모델/동작
    OPENAI_MODEL: str = "gpt-4o"
    TEMPERATURE: float = 0.5
    QUIZ_COUNT: int = 3
    QUIZ_COMMANDS: tuple[str, ...] = ("퀴즈", "퀴즈 시작")

    @computed_field
    @property
    def DATA_DIR(self) -> Path:
        path = self.BASE_DIR / "data"
        path.mkdir(parents=True, exist_ok=True)
        return path

    @computed_field
    @property
    def QUIZ_FILE(self) -> Path:
        return self.DATA_DIR / "quizzes.json"

    @computed_field
    @property
    def APPLICANT_FILE(self) -> Path:
        return self.DATA_DIR / "applicants.json"

    @computed_field
    @property
    def DB_FILE(self) -> Path:
        return self.DATA_DIR / "quiz_results.db"


# Settings 객체 생성
settings = Settings()

# 결과 출력
print(f"BASE_DIR: {settings.BASE_DIR}")
print(f"DATA_DIR: {settings.DATA_DIR}")
print(f"QUIZ_FILE: {settings.QUIZ_FILE}")
print(f"APPLICANT_FILE: {settings.APPLICANT_FILE}")
print(f"DB_FILE: {settings.DB_FILE}")

BASE_DIR: /home/lsmin/workspace/ai-agent/skala-agent-ext/src
DATA_DIR: /home/lsmin/workspace/ai-agent/skala-agent-ext/src/data
QUIZ_FILE: /home/lsmin/workspace/ai-agent/skala-agent-ext/src/data/quizzes.json
APPLICANT_FILE: /home/lsmin/workspace/ai-agent/skala-agent-ext/src/data/applicants.json
DB_FILE: /home/lsmin/workspace/ai-agent/skala-agent-ext/src/data/quiz_results.db


### `models.py`: 데이터 모델 정의

In [4]:
from typing import Literal, Optional
from pydantic import BaseModel, Field

class RoleRoute(BaseModel):
    """역할 기반 접근 제어를 위한 경로 모델입니다."""
    role: Literal["student", "professor", "unknown"]

class ApplicantInfo(BaseModel):
    """지원자 정보를 담는 클래스입니다."""
    student_class: str = Field(description="지원자의 학급")
    student_name: str = Field(description="지원자의 이름")
    student_id: str = Field(description="지원자의 학번")
    student_phone: str = Field(description="지원자의 전화번호")

class GradingResult(BaseModel):
    """단일 문제에 대한 채점 결과를 상세히 담는 클래스입니다."""
    question_id: int = Field(description="문제의 고유 ID")
    question: str = Field(description="채점 대상 문제")
    correct_answer: str = Field(description="문제의 정답")
    user_answer: str = Field(description="사용자가 제출한 답변")
    is_correct: bool = Field(description="정답 여부")
    explanation: str = Field(description="정답에 대한 친절한 해설")

class FinalReport(BaseModel):
    """퀴즈의 모든 채점 결과와 최종 점수를 종합한 최종 보고서 클래스입니다."""
    results: list[GradingResult] = Field(description="각 문제별 채점 결과 리스트")
    total_score: str = Field(description="'총점: X/Y' 형식의 최종 점수 요약")

class ReportRequest(BaseModel):
    """최종 보고서 생성을 위한 요청 모델입니다."""
    taken_date: Optional[str] = Field(None, description="YYYY-MM-DD 또는 YYYY.MM.DD")
    student_class: Optional[str] = Field(None, description="반 (예: '2반')")
    report_type: Literal["오답", "성적", "전체"] = "전체"


## 서비스 기능 구현

### `loaders.py`: 파일 데이터 로딩

In [5]:
import json, random
from quiz_agents.config import settings

def load_quizzes(count: int) -> list[dict]:
    """퀴즈 데이터를 JSON 파일에서 불러와 무작위로 `count`개를 선택합니다."""
    with open(settings.QUIZ_FILE, "r", encoding="utf-8") as f:
        all_q = json.load(f)
    return random.sample(all_q, min(count, len(all_q)))

def load_applicants() -> list[dict]:
    """지원자 데이터를 JSON 파일에서 불러옵니다."""    
    with open(settings.APPLICANT_FILE, "r", encoding="utf-8") as f:
        return json.load(f)


### `db.py`: DB 조작

**DB-API 2.0 (PEP 249) 호환 라이브러리** 는 두 가지 스타일을 지원

- `with closing(sqlite3.connect(settings.DB_FILE)) as conn` 패턴: 블록을 벗어나면 연결이 자동으로 닫힘

    ```python
    from contextlib import closing

    with closing(sqlite3.connect(settings.DB_FILE)) as conn:
        conn.execute("INSERT INTO quiz_results (student_name) VALUES (?)", ("홍길동",))
        conn.commit()

    ```

- `connection + cursor` 명시 선언 패턴: 여러 개의 cursor 동시 사용 가능, 개별적으로 close() 처리 필요

    ```python
    conn = sqlite3.connect(settings.DB_FILE)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO quiz_results (student_name) VALUES (?)", ("홍길동",))
    conn.commit()
    cursor.close()
    conn.close()
    ```

In [6]:
import sqlite3
import json
from contextlib import closing
from datetime import datetime

from quiz_agents.config import settings
from quiz_agents.models import FinalReport, ApplicantInfo, ReportRequest


def ensure_db():
    """데이터베이스 및 테이블 생성"""
    with closing(sqlite3.connect(settings.DB_FILE)) as conn:
        conn.execute("""
            CREATE TABLE IF NOT EXISTS quiz_results (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                taken_at TEXT NOT NULL,
                student_class TEXT,
                student_name TEXT,
                student_id TEXT,
                student_phone TEXT,
                total_score INTEGER,
                total_count INTEGER,
                details_json TEXT
            )
        """)
        conn.commit()


def save_report(applicant: ApplicantInfo, final_report: FinalReport):
    """퀴즈 결과 저장"""
    correct = sum(1 for r in final_report.results if r.is_correct)
    total = len(final_report.results)
    details = [r.model_dump() for r in final_report.results]

    with closing(sqlite3.connect(settings.DB_FILE)) as conn:
        conn.execute("""
            INSERT INTO quiz_results (
                taken_at, student_class, student_name, student_id, student_phone,
                total_score, total_count, details_json
            ) VALUES (?,?,?,?,?,?,?,?)
        """, (
            datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            applicant.student_class,
            applicant.student_name,
            applicant.student_id,
            applicant.student_phone,
            correct, total, json.dumps(details, ensure_ascii=False)
        ))
        conn.commit()

    return correct, total


def fetch_quiz_results(report_request: ReportRequest):
    """퀴즈 결과 조회"""
    taken_date = (report_request.taken_date or "").replace("/", "-").replace(".", "-")
    student_class = report_request.student_class or ""

    sql = """
        SELECT
            student_name, student_id, student_class,
            total_score, total_count, details_json, taken_at
        FROM quiz_results
        WHERE 1=1
    """
    params: list[str] = []

    if taken_date:
        sql += " AND taken_at LIKE ?"
        params.append(f"{taken_date}%")
    if student_class:
        sql += " AND student_class = ?"
        params.append(student_class)

    sql += " ORDER BY total_score DESC, taken_at ASC"

    with closing(sqlite3.connect(settings.DB_FILE)) as conn:
        rows = conn.execute(sql, params).fetchall()

    return rows


def fetch_quiz_applicant_taken(applicant: ApplicantInfo) -> dict | None:
    """응시자의 최근 퀴즈 결과 1건 조회 (있으면 taken_at/total_score 반환)"""
    query = """
        SELECT taken_at, total_score
        FROM quiz_results
        WHERE student_id = ?
        ORDER BY id DESC
        LIMIT 1
    """
    with closing(sqlite3.connect(settings.DB_FILE)) as conn:
        row = conn.execute(query, (applicant.student_id,)).fetchone()

    if row:
        taken_at, total_score = row
        return {"taken_at": taken_at, "total_score": total_score}
    return None


# 최초 실행 시 테이블 보장
ensure_db()
